In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from fuzzywuzzy import fuzz

In [2]:
gfd1 = pd.read_csv("...\\GFD-1\\FundObjective.txt", sep="\t", encoding='cp1252')
gfd1.head()

,LipperID,Objective
0,65006437,The fund is an equity fund geared towards achi...
1,65006438,The fund is an equity fund geared towards achi...
2,65006406,The equity fund aims to achieve long-term capi...
3,65006407,The equity fund aims to achieve long-term capi...
4,65012146,The equity fund aims to achieve long-term capi...


In [3]:
cfg = pd.read_csv("...\\CFG\\FundObjective.txt", sep="\t")
cfg_cols = cfg.columns
cfg.head()

,LipperID,Objective
0,60060059,The Goldman Sachs Global High Yield Portfolio ...
1,60060073,The Portfolio's investment objective is high t...
2,60067871,The fund is a mixed fund and aims to achieve m...
3,60068281,The investment objective of the fund is to gen...
4,60059801,The Goldman Sachs Global High Yield Portfolio ...


In [4]:
cfg.equals(gfd1)

False

In [5]:
print("Missing in GFD-1: " + str(len(cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]["LipperID"])))
missing_in_gfd1_df = cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]
missing_in_gfd1 = cfg[~cfg["LipperID"].isin(gfd1["LipperID"])]["LipperID"]


print("Missing in CFG: " + str(len(gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]["LipperID"])))
missing_in_cfg_df = gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]
missing_in_cfg = gfd1[~gfd1["LipperID"].isin(cfg["LipperID"])]["LipperID"]

universe_diffs = pd.concat([missing_in_cfg, missing_in_gfd1])

Missing in GFD-1: 122
Missing in CFG: 0


In [6]:
on=["LipperID"]
df = cfg.merge(gfd1, how='outer', on=on, suffixes=("_CFG", "_GFD-1"))
df = df[~df["LipperID"].isin(universe_diffs)]
df = df.fillna("-")
del cfg
del gfd1
df.sample(n=5)

,LipperID,Objective_CFG,Objective_GFD-1
29822,68243447,The Portfolio's investment objective is long t...,The Portfolio's investment objective is long t...
5317,65044759,The fund invests at least 70% of its assets in...,The fund invests at least 70% of its assets in...
7663,65124680,The investment objective of the Fund is to ach...,The investment objective of the Fund is to ach...
23462,68204216,The sub-fund seeks capital growth by investing...,The sub-fund seeks capital growth by investing...
4343,68324758,The objective of the fund is to provide a comb...,The objective of the fund is to provide a comb...


In [7]:
writer = pd.ExcelWriter("Fund Objective Checks.xlsx", engine='xlsxwriter')
df_dict = {}
missing_in_gfd1_df.to_excel(writer,"Additional in new feed", index=False)
missing_in_cfg_df.to_excel(writer,"Missing in new feed", index=False)
for col in cfg_cols[len(on):]:
    print(col+"\n")
    check=[]
    ratio = []
    columns = []
    columns = on + [col+"_CFG", col+"_GFD-1"]
    #print(columns)
    check_df = df.loc[:, columns]
    for n in range(df.shape[0]):
        ratio.append(fuzz.ratio(str(check_df.iloc[n, -2]),str(check_df.iloc[n, -1])))
        if str(check_df.iloc[n, -2]) == str(check_df.iloc[n, -1]):
            check.append("Match")
        elif str(check_df.iloc[n, -2]) == "-":
            check.append("Data not in CFG")
        elif str(check_df.iloc[n, -1]) == "-":
            check.append("Data not in GFD-1")
        else:
            check.append("Difference")
    check_df["Check"] = np.array(check)
    check_df["Ratio"] = np.array(ratio)
    check_df.drop_duplicates(inplace=True)
    check_df.to_excel(writer,'%s' % col, index=False)
    check_df[check_df.Check != "Match"].to_excel(writer,'%s Differences' % col, index=False)
    display(check_df.sample(n=5))
writer.save()

Objective



,LipperID,Objective_CFG,Objective_GFD-1,Check,Ratio
24983,68286205,The fund will invest most of its assets in equ...,The fund will invest most of its assets in equ...,Match,100
35624,68307995,The fund aims to achieve long-term returns thr...,<Not Available>,Difference,2
36406,68294029,The fund aims to deliver the return of the Gol...,The fund aims to deliver the return of the Gol...,Match,100
15445,68238600,The Sub-Fund aims to provide a positive absolu...,The Sub-Fund aims to provide a positive absolu...,Match,100
38690,68145152,The fund aims to achieve long-term capital gro...,The fund aims to achieve long-term capital gro...,Match,100
